<a href="https://colab.research.google.com/github/knolasco/Surfline-Object-Detection/blob/main/Wave_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

We are using Google Colab to apply transfer learning to this object detection project. Colab offers a free GPU that I can use to speed up the training process. From this notebook, I will extract the valuable model information and save it locally. Then, I will use a local jupyter notebook to run OpenCV.

# Install important packages

In [1]:
!pip install -U --pre tensorflow=="2.*"
!pip install tf_slim
!pip install pycocotools
import os
import pathlib

Requirement already up-to-date: tensorflow==2.* in /usr/local/lib/python3.7/dist-packages (2.4.1)
     |████████████████████████████████| 358kB 18.9MB/s 


# Make Folders for the Project

In [2]:
%%bash
mkdir Tensorflow
cd Tensorflow/
mkdir scripts
mkdir workspace
cd workspace
mkdir models
mkdir images
mkdir annotations
mkdir pre-trained-models
cd images
mkdir test
mkdir train

In [3]:
%%bash
cd Tensorflow
git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...


In [4]:
%%bash
cd Tensorflow/models/research/
protoc object_detection/protos/*.proto --python_out=.

In [5]:
%%bash
cd Tensorflow/models/research/
cp object_detection/packages/tf2/setup.py .

In [6]:
%%bash 
cd Tensorflow/models/research 
pip install .

Processing /content/Tensorflow/models/research
  Created wheel for object-detection: filename=object_detection-0.1-cp37-none-any.whl size=1625716 sha256=637c243a11ec44a26f497ef427c32a655b5d8480158236b5ebe2730698abc855
  Stored in directory: /tmp/pip-ephem-wheel-cache-9ezaqtuc/wheels/ac/8a/90/02d8042c0333b38a96341f55d93b379bb170f98b196de6b536
  Created wheel for avro-python3: filename=avro_python3-1.10.1-cp37-none-any.whl size=43735 sha256=650d12c80da9d22c6ca9b1c6ff3aa4e7e9db33fd575a11791a95936bdbd12327
  Stored in directory: /root/.cache/pip/wheels/65/fe/90/20d6d6d97223d80d20cb390be636619c536edab5658c12bdba
  Created wheel for future: filename=future-0.18.2-cp37-none-any.whl size=491058 sha256=c9adf85eccce29e2d582f4c7f834195a1de2bcc149104d9d8ee4199d00816135
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
  Created wheel for dill: filename=dill-0.3.1.1-cp37-none-any.whl size=78532 sha256=05cac3d99ae504ba5e33c88ac5a4e5cd0778d1b79

ERROR: multiprocess 0.70.11.1 has requirement dill>=0.3.3, but you'll have dill 0.3.1.1 which is incompatible.
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.25.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: apache-beam 2.28.0 has requirement avro-python3!=1.9.2,<1.10.0,>=1.8.1, but you'll have avro-python3 1.10.1 which is incompatible.


# Set up paths to the directories

In [7]:
WORKSPACE_PATH = 'Tensorflow/workspace'
SCRIPTS_PATH = 'Tensorflow/scripts'
APIMODEL_PATH = 'Tensorflow/models'
ANNOTATION_PATH = WORKSPACE_PATH +'/annotations'
IMAGE_PATH = WORKSPACE_PATH + '/images'
MODEL_PATH = WORKSPACE_PATH + '/models'
PRETRAINED_MODEL_PATH = WORKSPACE_PATH + '/pre-trained-models'
CONFIG_PATH = MODEL_PATH + '/my_ssd_mobnet/pipeline.config'
CHECKPOINT_PATH = MODEL_PATH + '/my_ssd_mobnet/'

# Create the label map

In [8]:
# these are the labels for the object detection
labels = [{'name' : 'left', 'id': 1},
             {'name': 'right', 'id' : 2}]

# create the file as pbtxt and save it to the annotation folder
with open(ANNOTATION_PATH + '/label_map.pbtxt', 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

# Create the TF Records

Before running the cell below, make sure to download the 'generate_tfrecord.py' file from here: https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/training.html#create-tensorflow-records

paste the file into the scripts folder

Also make sure the images are in the train/test folders


In [9]:
# this code is taken straight from the object detection API
!python /content/Tensorflow/scripts/generate_tfrecord.py -x /content/Tensorflow/workspace/images/train -l /content/Tensorflow/workspace/annotations/label_map.pbtxt -o /content/Tensorflow/workspace/annotations/train.record

!python /content/Tensorflow/scripts/generate_tfrecord.py -x /content/Tensorflow/workspace/images/test -l /content/Tensorflow/workspace/annotations/label_map.pbtxt -o /content/Tensorflow/workspace/annotations/test.record


Successfully created the TFRecord file: /content/Tensorflow/workspace/annotations/train.record
Successfully created the TFRecord file: /content/Tensorflow/workspace/annotations/test.record


# Copy the model config into the training folder

In [10]:
CUSTOM_MODEL_NAME = 'my_ssd_mobnet'

In [11]:
%%bash
cd Tensorflow/workspace/pre-trained-models/
mkdir checkpoint
mkdir saved_model
cd saved_model
mkdir variables

make sure to download the model from the model zoo before running the code below.

In [12]:
!mkdir {'Tensorflow/workspace/models/' + CUSTOM_MODEL_NAME}
!cp {PRETRAINED_MODEL_PATH + '/pipeline.config'} {MODEL_PATH + '/' + CUSTOM_MODEL_NAME}

# Update the config file for transfer learning

In [13]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [14]:
CONFIG_PATH = MODEL_PATH + '/' + CUSTOM_MODEL_NAME + '/pipeline.config'
config = config_util.get_configs_from_pipeline_file(CONFIG_PATH)

In [15]:
# create a shell for the pipeline config
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(CONFIG_PATH, 'r') as f:
    proto_str = f.read()
    text_format.Merge(proto_str, pipeline_config)

In [16]:
# edit the config to work for our project
pipeline_config.model.ssd.num_classes = 2
pipeline_config.train_config.batch_size = 32
pipeline_config.train_config.fine_tune_checkpoint = PRETRAINED_MODEL_PATH + '/checkpoint/ckpt-0' # edit this checkpoint in case I stop training early
pipeline_config.train_config.fine_tune_checkpoint_type = 'detection'
pipeline_config.train_input_reader.label_map_path = ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/train.record']
pipeline_config.eval_input_reader[0].label_map_path = ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/test.record']

In [17]:
pipeline_config

model {
  ssd {
    num_classes: 2
    image_resizer {
      fixed_shape_resizer {
        height: 320
        width: 320
      }
    }
    feature_extractor {
      type: "ssd_mobilenet_v2_fpn_keras"
      depth_multiplier: 1.0
      min_depth: 16
      conv_hyperparams {
        regularizer {
          l2_regularizer {
            weight: 3.9999998989515007e-05
          }
        }
        initializer {
          random_normal_initializer {
            mean: 0.0
            stddev: 0.009999999776482582
          }
        }
        activation: RELU_6
        batch_norm {
          decay: 0.996999979019165
          scale: true
          epsilon: 0.0010000000474974513
        }
      }
      use_depthwise: true
      override_base_feature_extractor_hyperparams: true
      fpn {
        min_level: 3
        max_level: 7
        additional_layer_depth: 128
      }
    }
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0
 

In [18]:
# input this pipeline_config into our project directory
config_text = text_format.MessageToString(pipeline_config)
with tf.io.gfile.GFile(CONFIG_PATH, 'wb') as f:
    f.write(config_text)

# Train the model

In [ ]:
# change --num_train_steps to 2000 to pick off where I left off
# make sure the model is downloaded from the zoo and add to /workspace/models/my_ssd_mobnet

In [19]:
with tf.device('/device:GPU:0'):
  !python /content/Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=/content/Tensorflow/workspace/models/my_ssd_mobnet --pipeline_config_path=/content/Tensorflow/workspace/models/my_ssd_mobnet/pipeline.config --num_train_steps=10000

2021-03-16 21:52:56.452566: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-03-16 21:52:59.082971: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-03-16 21:52:59.083820: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-03-16 21:52:59.090561: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-03-16 21:52:59.091244: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.75GiB deviceMemoryBandwidth: 298.08GiB/s
2021-03-16 21:52:59.091282: I tensorflow/stream_executor/platform/default/dso_loade

Things to grab from this file system:

* config from : /content/Tensorflow/workspace/models/my_ssd_mobnet/pipeline.config
* checkpoints from: /content/Tensorflow/workspace/models/my_ssd_mobnet
* label_map from: /content/Tensorflow/workspace/annotations/label_map.pbtxt
